### TODO

- Find the right way to turn the solution into integers

In [1]:
from random import randint
import sys

import numpy as np
import scipy.optimize as optimize
import pandas as pd
import matplotlib.pyplot as plt

plt.style.use("seaborn-whitegrid")
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
%xmode plain

def init_simulation(num_nodes, 
                    num_days, 
                    icu_capacities = None, 
                    transport_capacities = None, 
                    ini_path = None, 
                    demand_min = 80, 
                    demand_max = 90,
                    icu_min = 10,
                    icu_max = 300,
                    transport_min = 20,
                    transport_max = 30):
    if ini_path:
        print("you can also upload an .ini file")
        raise NotImplemented
    else:
        demand = []
        for node in range(num_nodes):
            demand.append(list(np.random.randint(demand_min, demand_max, size = num_days)))
        
        icu_capacities = icu_capacities if icu_capacities else list(np.random.randint(icu_min, 
                                                                                      icu_max, 
                                                                                      size = num_nodes))
        transport_capacities = transport_capacities if transport_capacities else list(np.random.randint(transport_min, 
                                                                                                        transport_max, 
                                                                                                        size = num_nodes)) 
    return icu_capacities, transport_capacities, demand


def calculate_outgoing(array,day,node):
    return sum(array[day][node])


def calculate_incoming(array,day,node):
    total_outgoing = 0
    for n in range(num_nodes):
        total_outgoing = total_outgoing + array[day][n][node]
    return total_outgoing


def demand_day_node_raw(node_capacity, demand_day, left_day, received_day, demand_previous_days, left_previous_days, received_previous_days):
    return demand_day - left_day + received_day+ min(demand_previous_days-left_previous_days+received_previous_days, node_capacity)


def demand_day_node(movements,day,node):
    movements_res = np.reshape(movements,(num_days,num_nodes,num_nodes))
    total_received_patients = 0
    total_outbound_patients = 0
    accumulated_demand = 0
    for d in range(day):
        total_received_patients = total_received_patients + calculate_incoming(movements_res,d,node)
        total_outbound_patients = total_outbound_patients + calculate_outgoing(movements_res,d,node)
    accumulated_demand = sum(demands[node][:day])
    
    demand = max(0, demand_day_node_raw(node_capacities[node],demands[node][day], 
                                        calculate_outgoing(movements_res,day,node), calculate_incoming(movements_res,day,node),
                                        accumulated_demand, total_outbound_patients, total_received_patients                
                ))
    return demand


def calc_total_deaths(movements):
    total = 0
    movements_res = np.reshape(movements,(num_days,num_nodes,num_nodes))
    for node in range(num_nodes):
        for day in range(num_days):
            deaths_not_attended = max(0, demand_day_node(movements,day,node) - node_capacities[node])
            deaths_transport = prob_death_transport*movements_res[day].sum()
            total = total + deaths_not_attended + deaths_transport
    return total


def outgoing_list(movements):
    movements_res = np.reshape(movements,(num_days,num_nodes,num_nodes))
    outgoing_list = list()
    for day in range(num_days):
        for node in range(num_nodes):
            outgoing_list.append(calculate_outgoing(movements_res,day,node))
    return outgoing_list

def generate_bounds():
    outgoing_list = list()
    for day in range(num_days):
        for outgoing_node in range(num_nodes):
            for incoming_node in range(num_nodes):
                outgoing_list.append((0,transport_capacities[outgoing_node]))
    return outgoing_list

def f_cons(node_id, day):
    return lambda x: transport_capacities[node_id] - calculate_outgoing(np.reshape(x,(num_days,num_nodes,num_nodes)),day ,node_id)


def split_n(num, num_nodes):
    #TODO this function is not truly random
    pieces = []
    for idx in range(num_nodes-1):
        pieces.append(randint(1,num-sum(pieces)-num_nodes+idx))

    pieces.append(num-sum(pieces))
    return pieces

Exception reporting mode: Plain


In [67]:
prob_death_transport = 0
num_days = 5
num_nodes = 4
node_capacities, transport_capacities, demands = init_simulation(num_nodes, 
                                                                 num_days, 
                                                                 icu_capacities=[3000, 200, 100, 50],
                                                                 transport_capacities=[0, 0, 0, 100], 
                                                                 demand_min = 60, demand_max=120)
print(f"""Randomly generated
{num_days} days, {num_nodes} hospitals
ICU capacities of hospitals: {node_capacities}
Daily transport capacities: {transport_capacities}, 
Daily simulated demand (from uniform distrution): 
{demands}""")

Randomly generated
5 days, 4 hospitals
ICU capacities of hospitals: [3000, 200, 100, 50]
Daily transport capacities: [0, 0, 0, 100], 
Daily simulated demand (from uniform distrution): 
[[80, 119, 113, 85, 118], [78, 92, 63, 67, 64], [99, 77, 77, 97, 63], [104, 111, 73, 108, 104]]


In [68]:
#define constraints
cons = []
for node in range(num_nodes):
    for day in range(num_days):
        cons.append({'type': 'ineq', 'fun': f_cons(node,day)})

        
#no movement matrix, the base for generating random ones and one of the ones to test as init matrix 
no_movement = np.array([[[0]*num_nodes]*num_nodes]*num_days)

In [69]:
def random_init(knockdown_days = False, movement_intensity = 1, random_intensity = False, prior_matrix = np.array([])): 
    random_movement = prior_matrix.copy() if prior_matrix.any() else no_movement.copy()
    #if knockdown days is active, behave like a dropdown NN layer and drop half of the days movements
    active_days = np.random.choice([1,0], size=num_days) if knockdown_days else [1]*num_days
    for day in range(num_days):
        if active_days[day]:
            for node_x in range(num_nodes):
                movement_intensity = np.random.uniform() if random_intensity else movement_intensity
                #splits a number in a list of numbers pseudo randomly
                split_nodes = split_n(max(num_nodes+1, 
                                          np.ceil(transport_capacities[node_x]*movement_intensity)), 
                                      num_nodes)
                for node_y in range(num_nodes):
                    if node_x != node_y: #the ones that move to "self" are actually staying
                        random_movement[day, node_x, node_y] += split_nodes[node_y]
                    
    return random_movement

### The main loop that generates random init matrices

In [74]:
%%time

#parameters to minimize
min_deaths = sys.maxsize
best_result = None
result_matrix = None
#iterate through low to high intensity movement simulation
steps = []
knockdown = True
win_init_matrix = no_movement.copy()
iterations = 50
movement_intensity = 0.5
for i in range(iterations):
    print(f"i:{i} ", end="")
    movement = random_init(knockdown, movement_intensity=movement_intensity, prior_matrix=win_init_matrix)
    bounds_movement = generate_bounds()
    result = optimize.minimize(calc_total_deaths, 
                               movement, 
                               method='SLSQP', 
                               constraints=cons, 
                               bounds=bounds_movement)

    current_deaths = calc_total_deaths(np.rint(result.x))
    print(f"Result:{current_deaths:.0f}", end="")
    steps.append([movement_intensity, knockdown, current_deaths])
    if min_deaths >= current_deaths: 
        print(" <--best!", end="")
        min_deaths = int(current_deaths)
        best_result = result
        result_matrix = np.reshape(np.rint(result.x),(num_days,num_nodes,num_nodes))
        win_init_matrix = movement

    print("")
        
print("\nDone")

i:0 Result:477 <--best!
i:1 Result:477 <--best!
i:2 Result:477 <--best!
i:3 Result:477 <--best!
i:4 Result:477 <--best!
i:5 Result:477 <--best!
i:6 Result:477 <--best!
i:7 Result:477 <--best!
i:8 Result:477 <--best!
i:9 Result:477 <--best!
i:10 Result:477 <--best!
i:11 Result:477 <--best!
i:12 Result:477 <--best!
i:13 Result:477 <--best!
i:14 Result:477 <--best!
i:15 Result:477 <--best!
i:16 Result:477 <--best!
i:17 Result:477 <--best!
i:18 Result:477 <--best!
i:19 Result:477 <--best!
i:20 Result:477 <--best!
i:21 Result:477 <--best!
i:22 Result:477 <--best!
i:23 Result:477 <--best!
i:24 Result:477 <--best!
i:25 Result:477 <--best!
i:26 Result:477 <--best!
i:27 Result:477 <--best!
i:28 Result:477 <--best!
i:29 Result:477 <--best!
i:30 Result:477 <--best!
i:31 Result:477 <--best!
i:32 Result:477 <--best!
i:33 Result:477 <--best!
i:34 Result:477 <--best!
i:35 Result:477 <--best!
i:36 Result:477 <--best!
i:37 Result:477 <--best!
i:38 Result:477 <--best!
i:39 Result:477 <--best!
i:40 Resul

### These are the deaths that we get with inaction

In [71]:
calc_total_deaths(no_movement)

927

### This is the best result found

In [72]:
min_deaths

477

### This the movement matrix

In [60]:
for node in range(num_nodes):
    for day in range(num_days):
        print(f'Node: {node}, Day: {day}, Demand: {demand_day_node(np.rint(result.x),day,node)}')

Node: 0, Day: 0, Demand: 312.0
Node: 0, Day: 1, Demand: 614.0
Node: 0, Day: 2, Demand: 919.0
Node: 0, Day: 3, Demand: 1141.0
Node: 0, Day: 4, Demand: 1399.0
Node: 1, Day: 0, Demand: 59.0
Node: 1, Day: 1, Demand: 62.0
Node: 1, Day: 2, Demand: 107.0
Node: 1, Day: 3, Demand: 114.0
Node: 1, Day: 4, Demand: 192.0
Node: 2, Day: 0, Demand: 0
Node: 2, Day: 1, Demand: 38.0
Node: 2, Day: 2, Demand: 55.0
Node: 2, Day: 3, Demand: 96.0
Node: 2, Day: 4, Demand: 98.0
Node: 3, Day: 0, Demand: 0
Node: 3, Day: 1, Demand: 9.0
Node: 3, Day: 2, Demand: 25.0
Node: 3, Day: 3, Demand: 37.0
Node: 3, Day: 4, Demand: 27.0
